In [18]:
import numpy as np
from numpy import expand_dims
from numpy import asarray
import os
from os import listdir
import cv2
import os
from PIL import Image
from mtcnn.mtcnn import MTCNN 
import pickle
import pandas as pd
from keras.models import load_model

In [19]:
faceDetector = MTCNN() 
FaceNetModel = load_model('facenet_keras.h5') #loading faceNet model

In [21]:
folder='Untitled Folder/'
database={}
for i in listdir(folder):
    
    image_path= folder + i
    image= Image.open(image_path) #reading image from the folder
    image = image.convert('RGB')
    image=asarray(image)
    mtcnn_results = faceDetector.detect_faces(image)
    # co-ordinates, height,width of detected face
    x1, y1, width, height = mtcnn_results[0]['box']
    x1=abs(x1)
    y1=abs(y1)
    x2=x1+width
    y2=y1+height 
    
    #cropping step
    face = image[y1:y2, x1:x2] 
    face = Image.fromarray(face)                       
    face = face.resize((160,160)) #faceNet uses this size
    face = asarray(face)
    
    #normalization
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    #faceNet expect 4 dimension in order to account for multiple face but we have on 1 face per image so input should be os 1x160x160x3
    face = expand_dims(face, axis=0)
    signature = FaceNetModel.predict(face)
    
    #storing database
    database[os.path.splitext(i)[0]]=signature


1/1 [==============================] - 0s 65ms/step


In [22]:
database

{'ashutosh': array([[-1.1555169 , -1.1593474 ,  0.6238339 ,  0.14198849, -1.1044692 ,
          1.9197496 , -0.1154779 ,  0.15041989, -0.19097298,  0.6527562 ,
          0.08398966,  0.43388036, -1.0216966 ,  0.7314658 ,  0.6821072 ,
         -1.0343668 ,  1.5468556 ,  0.33280277,  0.13987347,  0.01564491,
          0.9837872 , -2.2235246 ,  1.9430014 , -1.7737216 ,  0.5300961 ,
          0.9964862 , -1.0280936 ,  0.07727691,  0.00725515,  1.2255403 ,
          1.1191485 , -0.32040903,  0.5983333 , -1.0355715 ,  1.4085228 ,
          0.01847007, -0.9453788 ,  0.9981876 ,  0.8537731 ,  0.7092664 ,
         -0.80961967,  0.94321764,  1.441995  , -0.15333708,  1.1389551 ,
          1.7462078 , -1.0601869 ,  0.67627597, -0.7746685 ,  1.5985082 ,
          1.3124468 , -0.27223757,  0.8483662 , -1.3219622 ,  0.14888024,
          0.2567231 ,  0.11400661,  0.7246414 ,  0.35481974, -1.8928475 ,
          0.06576557,  0.43583107, -0.04334064, -0.14776063, -0.53591096,
         -1.5984052 , -1.8

In [23]:
#saving the database
with open('data.pkl', 'wb') as file:
    pickle.dump(database, file)